<a href="https://colab.research.google.com/github/tonytarizzo/NeverLateX/blob/main/Copy_of_STABILO_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import plot_model

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Attention**

❗❗❗❗❗Run the next two cells only if the corresponding files (X_train_merged.npy, X_test_merged.npy, y_train_merged.npy, y_test_merged.npy) have been deleted from /content/drive/My Drive/NeverLaTeX/AML. Otherwise, skip these two and run the next cell.❗❗❗❗❗


In [30]:
''' import os
import numpy as np

# Path to the main directory in Google Drive
base_path = "/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30"

# Initialize empty lists to hold the data
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

# Iterate over subdirectories
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):  # Ensure it's a directory
        try:
            # Load .npy files with allow_pickle=True
            X_train = np.load(os.path.join(folder_path, 'X_train.npy'), allow_pickle=True)
            X_test = np.load(os.path.join(folder_path, 'X_test.npy'), allow_pickle=True)
            y_train = np.load(os.path.join(folder_path, 'y_train.npy'), allow_pickle=True)
            y_test = np.load(os.path.join(folder_path, 'y_test.npy'), allow_pickle=True)

            # Append to respective lists
            X_train_list.append(X_train)
            X_test_list.append(X_test)
            y_train_list.append(y_train)
            y_test_list.append(y_test)
        except FileNotFoundError:
            print(f"Missing files in {folder_path}, skipping...")
        except Exception as e:
            print(f"Error processing {folder_path}: {e}")

# Concatenate all data
X_train_merged = np.concatenate(X_train_list, axis=0)
X_test_merged = np.concatenate(X_test_list, axis=0)
y_train_merged = np.concatenate(y_train_list, axis=0)
y_test_merged = np.concatenate(y_test_list, axis=0)

# Print shapes to verify
print(f"X_train_merged shape: {X_train_merged.shape}")
print(f"X_test_merged shape: {X_test_merged.shape}")
print(f"y_train_merged shape: {y_train_merged.shape}")
print(f"y_test_merged shape: {y_test_merged.shape}") '''

X_train_merged shape: (466404,)
X_test_merged shape: (159096,)
y_train_merged shape: (466404,)
y_test_merged shape: (159096,)


In [31]:
''' # Save merged data for future use
output_path = '/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30'
os.makedirs(output_path, exist_ok=True)

np.save(os.path.join(output_path, 'X_train_merged.npy'), X_train_merged, allow_pickle=True)
np.save(os.path.join(output_path, 'X_test_merged.npy'), X_test_merged, allow_pickle=True)
np.save(os.path.join(output_path, 'y_train_merged.npy'), y_train_merged, allow_pickle=True)
np.save(os.path.join(output_path, 'y_test_merged.npy'), y_test_merged, allow_pickle=True)

print(f"Merged datasets saved to {output_path}") '''

Merged datasets saved to /content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30


# **Attention**

❗❗❗❗❗Run the next cell if the corresponding files (X_train_merged.npy, X_test_merged.npy, y_train_merged.npy, y_test_merged.npy) exist in /content/drive/My Drive/NeverLaTeX/AML. Otherwise, skip this cell and run the previous two instead.❗❗❗❗❗

In [6]:
''' import os
import numpy as np

base_path = "/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30"

# Load .npy files with allow_pickle=True
X_train = np.load(os.path.join(base_path, 'X_train_merged.npy'), allow_pickle=True)
X_test = np.load(os.path.join(base_path, 'X_test_merged.npy'), allow_pickle=True)
y_train = np.load(os.path.join(base_path, 'y_train_merged.npy'), allow_pickle=True)
y_test = np.load(os.path.join(base_path, 'y_test_merged.npy'), allow_pickle=True) '''

In [25]:
''' # Load Subset of Data from Google Drive
data_path = "/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30/onhw2_both_indep_0"

X_train_path = f'{data_path}/X_train.npy'
X_test_path = f'{data_path}/X_test.npy'
y_train_path = f'{data_path}/y_train.npy'
y_test_path = f'{data_path}/y_test.npy' '''

In [26]:
''' # Load the data
import numpy as np

X_train = np.load(X_train_path, allow_pickle=True)
X_test = np.load(X_test_path, allow_pickle=True)
y_train = np.load(y_train_path, allow_pickle=True)
y_test = np.load(y_test_path, allow_pickle=True) '''

In [7]:
''' print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}") '''

X_train shape: (466404,)
X_test shape: (159096,)
y_train shape: (466404,)
y_test shape: (159096,)


In [8]:
''' lengths = [x.shape[0] for x in X_train]

print(f"Min: {min(lengths)}, Max: {max(lengths)}, Median: {np.median(lengths)}")

# Ensure no empty sequences exist
non_empty_indices = [i for i, x in enumerate(X_train) if x.shape[0] > 0]
X_train = X_train[non_empty_indices]
y_train = y_train[non_empty_indices]

non_empty_indices = [i for i, x in enumerate(X_test) if x.shape[0] > 0]
X_test = X_test[non_empty_indices]
y_test = y_test[non_empty_indices]

lengths = [x.shape[0] for x in X_train]

print(f"Min: {min(lengths)}, Max: {max(lengths)}, Median: {np.median(lengths)}") '''

Min: 0, Max: 3475, Median: 43.0
Min: 1, Max: 3475, Median: 43.0


In [9]:
''' from scipy.signal import resample

series_names = [
      'acc1x',
      'acc1y',
      'acc1z',
      'acc2x',
      'acc2y',
      'acc2z',
      'gyrox',
      'gyroy',
      'gyroz',
      'magnx',
      'magny',
      'magnz',
      'force',
  ]

def get_resampled_data(X, y, target_len=43):
    data_resampled = []
    new_y = []

    for item_idx, item in enumerate(X):
        if item.shape[0] == 0:  # Skip empty samples
            continue

        # Resample each feature across timesteps
        resampled_item = np.array([resample(item[:, series_idx], target_len) for series_idx in range(len(series_names))])

        # Transpose to shape (timesteps, features)
        resampled_item = resampled_item.T  # Shape: (target_len, 13)

        data_resampled.append(resampled_item)
        new_y.append(y[item_idx])

    # Convert lists to numpy arrays
    data_resampled = np.array(data_resampled)  # Shape: (samples, timesteps, features)
    new_y = np.array(new_y)  # Shape: (samples,)

    return data_resampled, new_y
 '''

In [11]:
''' X_train_resampled, y_train_resampled = get_resampled_data(X_train, y_train)
X_test_resampled, y_test_resampled = get_resampled_data(X_test, y_test)

output_path = '/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30'
os.makedirs(output_path, exist_ok=True)

np.save(os.path.join(output_path, 'X_train_resampled.npy'), X_train_resampled, allow_pickle=True)
np.save(os.path.join(output_path, 'X_test_resampled.npy'), X_test_resampled, allow_pickle=True)
np.save(os.path.join(output_path, 'y_train_resampled.npy'), y_train_resampled, allow_pickle=True)
np.save(os.path.join(output_path, 'y_test_resampled.npy'), y_test_resampled, allow_pickle=True)

print("Resampled datasets saved successfully.") '''

Resampled datasets saved successfully.


In [3]:
import os
import numpy as np

base_path = "/content/drive/My Drive/NeverLaTeX/AML/onhw-chars_2021-06-30"

# Load .npy files with allow_pickle=True
X_train = np.load(os.path.join(base_path, 'X_train_resampled.npy'), allow_pickle=True)
X_test = np.load(os.path.join(base_path, 'X_test_resampled.npy'), allow_pickle=True)
y_train = np.load(os.path.join(base_path, 'y_train_resampled.npy'), allow_pickle=True)
y_test = np.load(os.path.join(base_path, 'y_test_resampled.npy'), allow_pickle=True)

In [4]:
''' X_train = X_train_resampled
X_test = X_test_resampled
y_train = y_train_resampled
y_test = y_test_resampled
 '''
# Define input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])  # (timesteps, features)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Input shape for models:", input_shape)


X_train shape: (466357, 43, 13)
X_test shape: (159083, 43, 13)
Input shape for models: (43, 13)


In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the training and testing labels
all_labels = np.concatenate((y_train, y_test))  # Combine train and test labels for consistency
label_encoder.fit(all_labels)

# Transform the labels
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print(f"Classes: {label_encoder.classes_}")  # Display the mapping of characters to integers

# Convert labels to one-hot encoding

# Number of classes
num_classes = len(label_encoder.classes_)

# One-hot encode the labels
y_train_categorical = to_categorical(y_train_encoded, num_classes)
y_test_categorical = to_categorical(y_test_encoded, num_classes)

print(f"Shape of y_train: {y_train_categorical.shape}")
print(f"Shape of y_test: {y_test_categorical.shape}")

Classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j'
 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
Shape of y_train: (466357, 52)
Shape of y_test: (159083, 52)


In [6]:
# Define CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# Define CLDNN Model
from tensorflow.keras.layers import LSTM, BatchNormalization

def build_cldnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        LSTM(64, return_sequences=True),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
''' from re import X
# Train and Evaluate Models
print(X_train.shape)
print(X_test.shape)
print(y_train_categorical.shape)
print(y_test_categorical.shape, "\n")
input_shape = (X_train.shape[1], X_train.shape[2])  # Assuming (samples, timesteps, features)

# Ensure no empty sequences exist
non_empty_indices = [i for i, x in enumerate(X_train) if x.size > 0]
X_train = X_train[non_empty_indices]
y_train_categorical = y_train_categorical[non_empty_indices]

non_empty_indices = [i for i, x in enumerate(X_test) if x.size > 0]
X_test = X_test[non_empty_indices]
y_test_categorical = y_test_categorical[non_empty_indices]

print(X_train.shape)
print(X_test.shape)
print(y_train_categorical.shape)
print(y_test_categorical.shape)
 '''

(466357, 43, 13)
(159083, 43, 13)
(466357, 52)
(159083, 52) 

(466357, 43, 13)
(159083, 43, 13)
(466357, 52)
(159083, 52)


In [9]:
cnn_model = build_cnn(input_shape, num_classes)
cldnn_model = build_cldnn(input_shape, num_classes)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train CNN
cnn_history = cnn_model.fit(X_train, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test, y_test_categorical), verbose=1)

Epoch 1/50
5921/7287 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.0191 - loss: 43.4496

In [ ]:
# Train CLDNN
cldnn_history = cldnn_model.fit(X_train, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test, y_test_categorical), verbose=1)

In [ ]:
# Evaluate both models
cnn_loss, cnn_acc = cnn_model.evaluate(X_test, y_test, verbose=0)
cldnn_loss, cldnn_acc = cldnn_model.evaluate(X_test, y_test, verbose=0)

print(f"CNN Accuracy: {cnn_acc:.4f}, Loss: {cnn_loss:.4f}")
print(f"CLDNN Accuracy: {cldnn_acc:.4f}, Loss: {cldnn_loss:.4f}")

In [ ]:
# Visualize Results
import matplotlib.pyplot as plt

def plot_history(history, title):
    plt.figure()
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

plot_history(cnn_history, 'CNN Accuracy')
plot_history(cldnn_history, 'CLDNN Accuracy')